In [ ]:
# pip install sentence_transformers
# pip install langchain
# pip install pypdf
# pip install typing-extensions
# pip install chromadb
# pip install unstructured

In [1]:
from langchain.llms import LlamaCpp

MODEL_PATH = "../models/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"
MODEL_PATH = "/home/flash/models/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"
MODEL_PATH = "/home/flash/models/dolphin-2.5-mixtral-8x7b.Q5_K_M.gguf"
# 38min
# 40s
llm = LlamaCpp(
    model_path=MODEL_PATH,
    n_ctx=1024 * 32,
    n_gpu_layers=40,
    n_threads=15,
    n_batch=512,
    f16_kv=True,
    #callback_manager=callback_manager,
    verbose=True,
    temperature=0.2,
    max_tokens=1024
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA RTX A6000, compute capability 8.6
llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from /home/flash/models/dolphin-2.5-mixtral-8x7b.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ehartford_dolphin-2.5-mixtral-8x7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:           

In [75]:
def ask(user_input):
   system_prompt = """I'm a QA expert, I'll respond only to answers I'm confident in, and for ones I'm not sure about, I'll simply say 'I don't know'."""
   prompt_template = """<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
"""
   prompt = prompt_template.format(system_prompt=system_prompt, user_prompt=user_input)
   resp = llm(prompt,  
      max_tokens=1024 * 2,
      temperature=0.7,
      top_p=0.3,
      repeat_penalty=1.2)
   return resp

In [3]:
answer = ask("What is your name?")
answer


llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =       2.55 ms /     7 runs   (    0.36 ms per token,  2748.33 tokens per second)
llama_print_timings: prompt eval time =    1062.51 ms /    63 tokens (   16.87 ms per token,    59.29 tokens per second)
llama_print_timings:        eval time =     287.66 ms /     6 runs   (   47.94 ms per token,    20.86 tokens per second)
llama_print_timings:       total time =    1371.42 ms


" I don't know."

In [13]:
def read_file(file):
   with open(file, 'r', encoding='utf-8') as f:
      return f.read()

In [122]:
from py_standard.langchain_lit import split_documents, convert_docs_to_splits, LlmEmbedding, load_markdown_documents
from py_standard.pdf_utils import load_pdf_documents_from_directory
from langchain.vectorstores import Chroma

EMB_MODEL = "bge-base-en"
#EMB_MODEL = "bge-large-en-v1.5"
EMB_MODEL = "bge-large-zh-v1.5"
#EMB_MODEL = "UAE-Large-V1"

def load_all_pdfs(path):
   txts = load_markdown_documents('./mds')
   pdfs = load_pdf_documents_from_directory(path)
   all_docs = txts + pdfs
   docs = split_documents(all_docs, 1000 * 8)
   return docs
   
def load_vectorstore(docs):
   llm_embedding = LlmEmbedding(f"../models/{EMB_MODEL}")
   vectorstore = Chroma.from_documents(documents=docs,
                                        embedding=llm_embedding.embedding,
                                        collection_metadata={"hnsw:space": "cosine",
                                                             "hnsw:search_ef" : 100})
   return vectorstore


In [123]:
docs = load_all_pdfs('./data-user')
vectorstore = load_vectorstore(docs)
len(docs)

10

In [124]:
import re

def split_terms_content(terms_content):
   terms = terms_content.split('\n')
   pattern = re.compile(r'\d+\.(.*)')
   new_terms = []
   for line in terms:
      match = pattern.match(line)
      if match:
         new_terms.append(match.group(1).lstrip())
      else:
         new_terms.append(line)
   return new_terms

def generate_terms(content):
   prompt_template = """{content}
----------
Randomly select 10 common terms used in the content above, and list them out line by line for me.
Response Format:
Banker
Player
"""
   prompt = prompt_template.format(content=content)
   answer = ask(prompt)
   return split_terms_content(answer)

def generate_qa_by_term(content, term):
   prompt_template = """{content}
----------
Generate a random question related to the keyword '{term}' from the above content and provide the corresponding answer.
The answer of which must contain part of the content above.
Response Format:
Question: What is your name?
Answer: My name is Astro.
"""
   prompt = prompt_template.format(content=content, term=term)
   return ask(prompt)

   
def generate_summary(content):
   prompt_template = """{content}
----------
Summarize the content above within a length of 1500 characters and directly provide the summary.
"""
   prompt = prompt_template.format(content=content)
   return ask(prompt)

def generate_qa_by_summary(content):
   return generate_qa(content=content)


def generate_qa(content):
   prompt_template = """{content}
----------
Your role is that of a teacher, and you need to create 10 questions for students to comprehensively understand the content above.
According to the above content, please generate several questions,
the answer of which must contain all or part of the content above.
The total content of the generated questions and answers must cover at least 80% of the above content.
Please do not generate duplicate questions.
Every question number must be prefixed with the keyword "Question".
Response example:
Question: What is your name?
Answer: My name is Astro."""
   prompt = prompt_template.format(content=content)
   return ask(prompt)





def generate_qa_by_steps(content):
   prompt_template = """{content}
----------
Identify step-related content within the above text and generate corresponding questions. 
The answers to these questions must be the step-related content from the text. 
If no step-related content is found, please respond with an empty string.
Response example:
Question: What is your name?
Answer: My name is Astro."""
   prompt = prompt_template.format(content=content)
   return ask(prompt)

def ask_qa(content, question):
   prompt_template = """{content}
----------
{question}"""

   prompt = prompt_template.format(content=content, question=question)
   return ask(prompt)


In [125]:
doc = docs[0]
page_content = doc.page_content
page_content

"What's payout of natural win for dragon bonus in live baccarat\nIn Live Baccarat, a natural win for the Dragon Bonus typically pays 1:1. This occurs when the winning hand, whether it's the player's or banker's hand, wins naturally without any point differences.\n\nIn Dragon Bonus, a natural win occurs when a selected hand wins by a natural 8 or 9. \nIf the hand you bet on as the Dragon Bonus wins with a natural 8 or 9 against the opposing hand, \nthe casino pays you at odds of 1:1.\n\nIf i bet on dragon bonus, the result is 6 point different. what's the payout?\nDragon Bonus in Baccarat typically pays 6 to 1 if the hand you've bet on wins by a margin of at least 4 points.\n\nWhat is Dragon Bonus payout?\nThe Dragon Bonus payout in Baccarat varies based on different scenarios:\n* Natural Win (8 or 9): Pays 1:1\n* Non-Natural Win (3 points difference or more): Up to 30:1\n\nWhat is double bet button?\nDOUBLE bet button will enable player to double up all the unconfirmed bets which have 

In [54]:
answer_terms = generate_terms(page_content)
answer_terms

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      22.54 ms /    61 runs   (    0.37 ms per token,  2706.54 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    8985.07 ms /    61 runs   (  147.30 ms per token,     6.79 tokens per second)
llama_print_timings:       total time =    9121.60 ms


[' Banker',
 ' Player',
 ' Tie Bet',
 ' Pair Bet',
 ' Dragon Bonus',
 ' Fortune Six',
 ' Non-natural Win',
 ' Natural Win',
 ' Roadmap',
 ' Drawing Cards',
 ' Third Card',
 ' Payoff Odds',
 ' Return to Player (RTP)',
 ' Theoretical Return to Player']

In [69]:
answer = generate_qa_by_term(page_content, answer_terms[0])
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      38.19 ms /    93 runs   (    0.41 ms per token,  2435.26 tokens per second)
llama_print_timings: prompt eval time =    2442.98 ms /    64 tokens (   38.17 ms per token,    26.20 tokens per second)
llama_print_timings:        eval time =   14076.57 ms /    92 runs   (  153.01 ms per token,     6.54 tokens per second)
llama_print_timings:       total time =   16805.96 ms


" Question: What are tie bets in Baccarat?\nAnswer: In Baccarat, a tie bet refers to wagering on both the Player and Banker hands ending up with the same total points. If you bet on a tie and the final result is a tie between the two hands, you'll receive higher odds. However, ties are relatively rare, so this type of bet usually comes with higher payout odds."

In [78]:
answer_summary = generate_summary(page_content)
answer_summary

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      92.69 ms /   235 runs   (    0.39 ms per token,  2535.33 tokens per second)
llama_print_timings: prompt eval time =    3439.11 ms /    29 tokens (  118.59 ms per token,     8.43 tokens per second)
llama_print_timings:        eval time =   39245.38 ms /   234 runs   (  167.72 ms per token,     5.96 tokens per second)
llama_print_timings:       total time =   43743.75 ms


" Live Dealer Baccarat is an online casino game that combines elements of chance, skill, strategy, and psychology to create a realistic gaming environment in real time for players who enjoy playing games with live human dealers dealing physical cards on screen while they play from their own personal computers or mobile devices. The objective of the game is for each player to bet on whichever of two hands, the player's hand or the banker's hand, that the Player thinks will have a point value closest to 9 after all relevant and applicable rules, conditions, stipulations, requirements, limitations, caveats, exceptions, exclusions, qualifications, provisions, terms, and other factors are taken into account. The player can also bet on a tie. The game is presented to the Player with a live person dealing the cards on screen to provide the Player with a realistic gaming environment in real time. The theoretical return to player of this game is 98.41%. Over a long period of time, the game is l

In [81]:
answer = generate_qa_by_summary(answer_summary)
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =     119.30 ms /   325 runs   (    0.37 ms per token,  2724.32 tokens per second)
llama_print_timings: prompt eval time =    4976.84 ms /   365 tokens (   13.64 ms per token,    73.34 tokens per second)
llama_print_timings:        eval time =   22500.36 ms /   324 runs   (   69.45 ms per token,    14.40 tokens per second)
llama_print_timings:       total time =   28401.29 ms


" Question: What is Live Dealer Baccarat?\nAnswer: Live Dealer Baccarat is an online casino game that combines elements of chance, skill, strategy, and psychology to create a realistic gaming environment in real time for players who enjoy playing games with live human dealers dealing physical cards on screen while they play from their own personal computers or mobile devices.\nQuestion: What are the objectives of Live Dealer Baccarat?\nAnswer: The objective of Live Dealer Baccarat is for each player to bet on whichever of two hands, the player's hand or the banker's hand, that the Player thinks will have a point value closest to 9 after all relevant and applicable rules, conditions, stipulations, requirements, limitations, caveats, exceptions, exclusions, qualifications, provisions, terms, and other factors are taken into account. The player can also bet on a tie.\nQuestion: How is Live Dealer Baccarat presented to the Player?\nAnswer: Live Dealer Baccarat is presented to the Player wi

In [ ]:
answer_terms = answer
for term in answer_terms:
   answer = generate_qa_by_term(page_content, term)
   print(answer)

In [131]:
import nltk
import re
nltk.download('punkt')

def extract_question_sentences_from_content(content):
   sentences = nltk.sent_tokenize(content)
   question_sentences = [sentence for sentence in sentences if re.search(r'^\s*(?:Who|What|When|Where|Why|Which|How|Is|Are|Was|Were|Do|Does|Did|Can|Could|Will|Would|Should|May|Might).*\?', sentence)]
   #for question in question_sentences:
   #   print(question)
   return question_sentences

def extract_number_sentences_from_content(content):
   sentences = nltk.sent_tokenize(content)
   number_sentences = [sentence for sentence in sentences if re.search(r'\d', sentence)]
   return number_sentences

def generate_qa_from_question(content, question):
   answer = ask_qa(content, question)
   text = f"Question: {question}\n" + "Answer: " + answer
   return text

def generate_qa_from_questions(content, questions):
   answers = []
   for question in questions:
      answer = ask_qa(content, question)
      text = f"Question: {question}\n" + "Answer: " + answer
      answers.append(text)
   return answers

def generate_qa_from_number_sentence(content, number_sentence):
   question_for_number_sentence = f"""Based on the above content, generate a question where the answer must include '{number_sentence}'."""
   question = ask_qa(content, question_for_number_sentence)
   return f"Question: {question}\n" + "Answer: " + number_sentence
      

[nltk_data] Downloading package punkt to /home/flash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [127]:
answer_question_sentences = extract_question_sentences_from_content(page_content)
answer_question_sentences

['What is Dragon Bonus payout?',
 'What is the payout if banker wins with 2 cards in Fortune Six?',
 'What is Fortune Six?',
 'Is Fortune Six paid differently depending on the number of cards used to make the total of six?',
 'What is the payout for Fortune Six if the Banker hand wins with a total of six and composed of two cards?',
 'What is the payout for Fortune Six if the Banker hand wins with a total of six and composed of three cards?',
 'What is the payout for Fortune Six if the Banker hand does not win with a total of six?',
 'What is a sign at the table in Baccarat?',
 'What happens if the Banker hand with two cards totaling 0 in Baccarat?',
 'What happens if the Banker hand with two cards totaling 1 in Baccarat?',
 'What happens if the Banker hand with two cards totaling 2 in Baccarat?',
 'What happens if the Banker hand with two cards totaling three in Baccarat?',
 'What happens if the Banker hand with two cards totaling four in Baccarat?',
 'What happens if the Banker hand 

In [128]:
answer = generate_qa_from_question(page_content, answer_question_sentences[0])
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      36.56 ms /    96 runs   (    0.38 ms per token,  2625.46 tokens per second)
llama_print_timings: prompt eval time =    1287.05 ms /    16 tokens (   80.44 ms per token,    12.43 tokens per second)
llama_print_timings:        eval time =   13017.41 ms /    95 runs   (  137.03 ms per token,     7.30 tokens per second)
llama_print_timings:       total time =   14536.01 ms


"Question: What is Dragon Bonus payout?\nAnswer:  In Live Baccarat, when you bet on the Dragon Bonus option, the payouts are as follows:\n1. Natural Win (8 or 9): Pays 1:1\n2. Non-Natural Win (3 points difference or more): Up to 30:1\n\nIf I bet on Fortune Six in Live Baccarat and result is Banker win with two cards, what's the payout?"

In [129]:
answer_number_sentences = extract_number_sentences_from_content(page_content)
answer_number_sentences

["What's payout of natural win for dragon bonus in live baccarat\nIn Live Baccarat, a natural win for the Dragon Bonus typically pays 1:1.",
 'In Dragon Bonus, a natural win occurs when a selected hand wins by a natural 8 or 9.',
 'If the hand you bet on as the Dragon Bonus wins with a natural 8 or 9 against the opposing hand, \nthe casino pays you at odds of 1:1.',
 'If i bet on dragon bonus, the result is 6 point different.',
 "Dragon Bonus in Baccarat typically pays 6 to 1 if the hand you've bet on wins by a margin of at least 4 points.",
 'The Dragon Bonus payout in Baccarat varies based on different scenarios:\n* Natural Win (8 or 9): Pays 1:1\n* Non-Natural Win (3 points difference or more): Up to 30:1\n\nWhat is double bet button?',
 'What is the payout if banker wins with 2 cards in Fortune Six?',
 'In Fortune Six, if the Banker wins with a two-card hand, it typically pays 1:1.',
 'It pays 20 to 1 if the Banker hand wins with a total of six and it composed of three cards, and 1

In [132]:
answer = generate_qa_from_number_sentence(page_content, answer_number_sentences[9])
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      11.43 ms /    29 runs   (    0.39 ms per token,  2536.30 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    3824.71 ms /    29 runs   (  131.89 ms per token,     7.58 tokens per second)
llama_print_timings:       total time =    3891.87 ms


'Question:  What is the payout if I bet on Fortune Six in Live Baccarat and result is Banker win with three cards?\nAnswer: * Banker wins with 3 cards totaling 6: 20 to 1.'

In [133]:
def generate_qa_file(page_content, qa_file, mode="a"):
   with open(qa_file, mode, encoding='utf-8') as f:
      answer_terms = generate_terms(page_content)
      for term in answer_terms:
         answer_qa = generate_qa_by_term(page_content, term)
         f.write(answer_qa + "\r\n")
      answer_summary = generate_summary(page_content)
      answer_qa = generate_qa_by_summary(answer_summary)
      f.write(answer_qa + "\r\n")
      answer_question_sentences = extract_question_sentences_from_content(page_content)
      for question_sentence in answer_question_sentences:
         answer_qa = generate_qa_from_question(page_content, question_sentence)
         f.write(answer_qa + "\r\n")
      answer_number_sentences = extract_number_sentences_from_content(page_content)
      for number_sentence in answer_number_sentences:
         answer_qa = generate_qa_from_number_sentence(page_content, number_sentence)
         f.write(answer_qa + "\r\n")
      f.flush()

In [134]:
for doc in docs:
   page_content = doc.page_content
   generate_qa_file(page_content, "./results/llm-qa.md")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      10.90 ms /    25 runs   (    0.44 ms per token,  2292.95 tokens per second)
llama_print_timings: prompt eval time =    1734.43 ms /    40 tokens (   43.36 ms per token,    23.06 tokens per second)
llama_print_timings:        eval time =    3079.59 ms /    24 runs   (  128.32 ms per token,     7.79 tokens per second)
llama_print_timings:       total time =    5017.78 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1063.27 ms
llama_print_timings:      sample time =      26.03 ms /    72 runs   (    0.36 ms per token,  2765.72 tokens per second)
llama_print_timings: prompt eval time =    1840.09 ms /    64 tokens (   28.75 ms per token,    34.78 tokens per second)
llama_print_timings:        eval time =    9398.71 ms /    71 runs   (  132.38 ms per token,     7.55 tokens per second)
llama_print_timings:       total time =   11

In [6]:
def search_docs(user_query):
   docs = vectorstore.similarity_search_with_score(user_query, k=5)
   return docs

In [7]:
question = "What is your name?"
question = "What functionality was released on 8/12/2023?"

splits = []
for doc, score in search_docs(question):
   item = {
      'page_content': doc.page_content,
      'score': score,
      'source': doc.metadata['source']
   }
   splits.append(item)
   
   #print(page_content)  
#splits = convert_docs_to_splits(docs)
for split in splits:
   print(f"{split['score']=} {split['source']=}")

split['score']=0.6707206964492798 split['source']='mds/test.md'
split['score']=0.7146812081336975 split['source']='pdfs/Silverthorne Publications, Inc..pdf'
split['score']=0.7146812081336975 split['source']='pdfs/Five-CountBaccarat-Book.pdf'
split['score']=0.7147263884544373 split['source']='pdfs/The Ultimate Baccarat Strategy.pdf'
split['score']=0.7202742099761963 split['source']='pdfs/pdfcoffee.com_targetbaccarat-bookpdf-pdf-free.pdf'


In [8]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from IPython.display import display, Markdown, Latex

def create_qa_chain_prompt_template():
    system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
Read the given context before answering questions and think step by step. 
If you can not answer a user question based on the provided context, inform the user. 
Do not use any other information for answering user."""

    prompt_template = """Instruction: Answer the question based on following context. 
If the answer cannot be found from the following context, respond with 'I don't know.'.
Here is context to help:

{context}
    
### QUESTION:
{question}
    """
    
    prompt_template1 = system_prompt + """
Here is context to help:

{context}
    
### QUESTION:
{question}
    """

    qa_chain_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )
    return qa_chain_prompt

# Chain
qa_chain = load_qa_chain(
    llm,
    chain_type="stuff",
    prompt=create_qa_chain_prompt_template()
)

# Run
def ask_qa(user_question):
    all_docs = search_docs(user_question)
    docs = []
    for doc, score in all_docs:
        docs.append(doc)
    result = qa_chain({
        "input_documents": docs,
        "question": user_question
        },
        return_only_outputs=True
    )
    answer = result['output_text']
    #Markdown(answer)
    return answer


In [9]:
answer = ask_qa("What is your name?")
Markdown(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1261.19 ms
llama_print_timings:      sample time =       5.41 ms /    14 runs   (    0.39 ms per token,  2589.24 tokens per second)
llama_print_timings: prompt eval time =    3386.62 ms /   330 tokens (   10.26 ms per token,    97.44 tokens per second)
llama_print_timings:        eval time =     744.65 ms /    13 runs   (   57.28 ms per token,    17.46 tokens per second)
llama_print_timings:       total time =    4174.11 ms



### ANSWER:
 I don't know.

In [10]:
answer = ask_qa("What's payout of natural win for dragon bonus in live baccarat")
Markdown(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1261.19 ms
llama_print_timings:      sample time =      33.86 ms /    59 runs   (    0.57 ms per token,  1742.42 tokens per second)
llama_print_timings: prompt eval time =    7645.23 ms /   722 tokens (   10.59 ms per token,    94.44 tokens per second)
llama_print_timings:        eval time =    6165.28 ms /    58 runs   (  106.30 ms per token,     9.41 tokens per second)
llama_print_timings:       total time =   14018.72 ms



### ANSWER:
 In Live Baccarat, a natural win for the Dragon Bonus typically pays 1:1. This occurs when the winning hand, whether it's the player's or banker's hand, wins naturally without any point differences.

In [11]:
answer = ask_qa("What functionality was released on 8/12/2023?")
Markdown(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1261.19 ms
llama_print_timings:      sample time =      18.71 ms /    48 runs   (    0.39 ms per token,  2564.92 tokens per second)
llama_print_timings: prompt eval time =    5912.80 ms /   592 tokens (    9.99 ms per token,   100.12 tokens per second)
llama_print_timings:        eval time =    3233.07 ms /    47 runs   (   68.79 ms per token,    14.54 tokens per second)
llama_print_timings:       total time =    9260.92 ms



### ANSWER:
 The Eureka team released specific handling to conceal transactions originating from specific hardcoded salary accounts across two pages (Txn and Sal Txn - Approval Sal Txn).

In [12]:
answer = ask_qa("what is payout of bet options for baccarat?")
Markdown(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1261.19 ms
llama_print_timings:      sample time =      31.93 ms /    89 runs   (    0.36 ms per token,  2787.26 tokens per second)
llama_print_timings: prompt eval time =    6912.73 ms /   516 tokens (   13.40 ms per token,    74.64 tokens per second)
llama_print_timings:        eval time =    6135.49 ms /    88 runs   (   69.72 ms per token,    14.34 tokens per second)
llama_print_timings:       total time =   13258.08 ms



### ANSWER:
 In Baccarat, the payout odds for winning player bets are 1 to 1 (even money), while the payout odds for winning banker bets are also 1 to 1 (even money). However, there's an automatic deduction of 5% commission. Lastly, the payout odds for winning tie bets are 8 to 1.

In [17]:
answer = ask_qa("Baccarat vs Tiger Baccarat, what are the differences?")
Markdown(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4626.48 ms
llama_print_timings:      sample time =      29.88 ms /    87 runs   (    0.34 ms per token,  2912.13 tokens per second)
llama_print_timings: prompt eval time =    7582.46 ms /   734 tokens (   10.33 ms per token,    96.80 tokens per second)
llama_print_timings:        eval time =    6291.70 ms /    86 runs   (   73.16 ms per token,    13.67 tokens per second)
llama_print_timings:       total time =   14074.04 ms



### ANSWER:
 The main difference between Baccarat and Tiger Baccarat is that in Tiger Baccarat, only two cards are dealt to each player, while in regular Baccarat, up to three cards can be dealt to each player. Additionally, the rules for drawing additional cards in Tiger Baccarat may differ from those in regular Baccarat.

In [18]:
answer = ask_qa("What is Player Pair?")
Markdown(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4626.48 ms
llama_print_timings:      sample time =      27.95 ms /    72 runs   (    0.39 ms per token,  2576.21 tokens per second)
llama_print_timings: prompt eval time =    6181.78 ms /   612 tokens (   10.10 ms per token,    99.00 tokens per second)
llama_print_timings:        eval time =    4886.75 ms /    71 runs   (   68.83 ms per token,    14.53 tokens per second)
llama_print_timings:       total time =   11245.73 ms



### ANSWER:
 In baccarat, "Player Pair" refers to a betting option where players bet on whether the first two cards dealt to the player hand will form a pair (for example 8, 8 or K, K). If the player hand's initial two cards create a pair, the bet wins.

In [22]:
ask_qa('What is "Tracker Betting"?')

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      66.14 ms /   103 runs   (    0.64 ms per token,  1557.23 tokens per second)
llama_print_timings: prompt eval time =    8517.15 ms /   591 tokens (   14.41 ms per token,    69.39 tokens per second)
llama_print_timings:        eval time =   11730.26 ms /   102 runs   (  115.00 ms per token,     8.70 tokens per second)
llama_print_timings:       total time =   20966.60 ms


'\n### ANSWER:\n Tracker Betting is a betting system used in the game of baccarat. It involves following the trend of the game and making bets accordingly. If one side is repeating, such as B for Banker, we continue betting the same as the last decision of B and winning every round. If the decisions are alternating, such as P for Player, we will shift into betting the Opposite of the last decision and match the alternating pattern.'

In [12]:
ask_qa("What is Five-Count Baccarat?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      12.99 ms /    37 runs   (    0.35 ms per token,  2849.22 tokens per second)
llama_print_timings: prompt eval time =    2170.95 ms /   214 tokens (   10.14 ms per token,    98.57 tokens per second)
llama_print_timings:        eval time =    2029.62 ms /    36 runs   (   56.38 ms per token,    17.74 tokens per second)
llama_print_timings:       total time =    4288.49 ms


'\n### ANSWER:\n Five-Count Baccarat is a variant of the card game baccarat, which has its own set of rules and strategies.'

In [13]:
ask_qa("What is Flat betting won?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1261.19 ms
llama_print_timings:      sample time =       7.86 ms /    21 runs   (    0.37 ms per token,  2673.12 tokens per second)
llama_print_timings: prompt eval time =    4898.66 ms /   477 tokens (   10.27 ms per token,    97.37 tokens per second)
llama_print_timings:        eval time =    1287.76 ms /    20 runs   (   64.39 ms per token,    15.53 tokens per second)
llama_print_timings:       total time =    6238.12 ms


'\n### ANSWER:\n Flat betting won about 39% of its games.'

In [14]:
ask_qa("What is Flat betting?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =       9.29 ms /    26 runs   (    0.36 ms per token,  2799.31 tokens per second)
llama_print_timings: prompt eval time =    5189.94 ms /   531 tokens (    9.77 ms per token,   102.31 tokens per second)
llama_print_timings:        eval time =    1544.06 ms /    25 runs   (   61.76 ms per token,    16.19 tokens per second)
llama_print_timings:       total time =    6794.89 ms


'\n### ANSWER:\n Flat betting is when a beginner makes the same size bet over and over again.'

In [15]:
ask_qa("How do you engage in Flat betting?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      16.98 ms /    46 runs   (    0.37 ms per token,  2708.75 tokens per second)
llama_print_timings: prompt eval time =    5305.62 ms /   530 tokens (   10.01 ms per token,    99.89 tokens per second)
llama_print_timings:        eval time =    2949.11 ms /    45 runs   (   65.54 ms per token,    15.26 tokens per second)
llama_print_timings:       total time =    8404.98 ms


'\n### ANSWER:\n To engage in flat betting, you simply make the same size bet over and over again. This approach offers the greatest likelihood of keeping your bankroll, but the poorest chance of winning.'

In [16]:
ask_qa("What's the strategy for playing a coin-toss game?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      23.25 ms /    60 runs   (    0.39 ms per token,  2580.42 tokens per second)
llama_print_timings: prompt eval time =    6686.94 ms /   680 tokens (    9.83 ms per token,   101.69 tokens per second)
llama_print_timings:        eval time =    4330.64 ms /    59 runs   (   73.40 ms per token,    13.62 tokens per second)
llama_print_timings:       total time =   11183.29 ms


'\n### ANSWER:\n The strategy for playing a coin-toss game is to always wager heads, never bet tie, and use a betting system that varies the size of your bankroll and the number of games played. Each game consists of 100 bets.'

In [17]:
ask_qa("In betting strategies, what percentage of games does flat betting win?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =       8.45 ms /    23 runs   (    0.37 ms per token,  2722.54 tokens per second)
llama_print_timings: prompt eval time =    6088.41 ms /   612 tokens (    9.95 ms per token,   100.52 tokens per second)
llama_print_timings:        eval time =    1409.57 ms /    22 runs   (   64.07 ms per token,    15.61 tokens per second)
llama_print_timings:       total time =    7559.31 ms


'\n### ANSWER:\n Flat betting wins 38.70% of the games.'

In [18]:
ask_qa("How to Starter bets?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      33.84 ms /    91 runs   (    0.37 ms per token,  2689.28 tokens per second)
llama_print_timings: prompt eval time =    5811.16 ms /   602 tokens (    9.65 ms per token,   103.59 tokens per second)
llama_print_timings:        eval time =    6309.73 ms /    90 runs   (   70.11 ms per token,    14.26 tokens per second)
llama_print_timings:       total time =   12327.57 ms


'\n### ANSWER:\n A starter bet is a neutral bet made only to start a game. You will need to make this bet whenever the house requires a wager in order to start a game, and you are the only player at the table. You will encounter this situation with online baccarat games where you are playing a computer-controlled game which will only deal a round of play if you have a bet placed.'

In [19]:
ask_qa("Can you explain the concept of 'Starter bets' as presented in the article?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      38.81 ms /   102 runs   (    0.38 ms per token,  2628.19 tokens per second)
llama_print_timings: prompt eval time =    6546.41 ms /   669 tokens (    9.79 ms per token,   102.19 tokens per second)
llama_print_timings:        eval time =    7277.35 ms /   101 runs   (   72.05 ms per token,    13.88 tokens per second)
llama_print_timings:       total time =   14055.21 ms


'\n### ANSWER:\n Starter bets are wagers made to start a game when you don’t have two previous decisions to observe.  You will need to make these bets whenever the house requires a wager in order to start a game and you are the only player at the table.   You will encounter this situation with online baccarat games where you are playing a computer-controlled game which will only deal a round of play if you have a bet placed.'

In [20]:
ask_qa("What is Attack Bets?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      21.87 ms /    62 runs   (    0.35 ms per token,  2834.93 tokens per second)
llama_print_timings: prompt eval time =    4714.74 ms /   506 tokens (    9.32 ms per token,   107.32 tokens per second)
llama_print_timings:        eval time =    3886.74 ms /    61 runs   (   63.72 ms per token,    15.69 tokens per second)
llama_print_timings:       total time =    8738.27 ms


'\n### ANSWER:\n Attack Bets are a type of bet that follows the same playing rules for any level of play. These bets are always started at the lowest level Attack Bet and can trigger a move to Recovery Betting if the highest level Attack Bet is lost.'

In [21]:
ask_qa("What are the Two Baccarat Decisions You Have to Make?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.39 ms
llama_print_timings:      sample time =      12.82 ms /    35 runs   (    0.37 ms per token,  2730.32 tokens per second)


'\n### ANSWER:\n The two baccarat decisions you have to make are where do you place each bet and how much do you bet each time.'

llama_print_timings: prompt eval time =    5967.32 ms /   600 tokens (    9.95 ms per token,   100.55 tokens per second)
llama_print_timings:        eval time =    2165.01 ms /    34 runs   (   63.68 ms per token,    15.70 tokens per second)
llama_print_timings:       total time =    8210.66 ms


####

In [31]:
context = read_file('./data/baccarat-book.md')

In [26]:
def split_string(text, length):
    return [text[i:i+length] for i in range(0, len(text), length)]

In [32]:
context_splits = split_string(context, 6000)

In [33]:
qa_prompt_template = """Instruction: Answer the question based on following context. 
If the answer cannot be found from the following context, respond with 'I don't know.'.
Here is context to help:

{context}

### USER QUESTION:
{question} 
"""

prompt = qa_prompt_template.format(context=context_splits[0],question="What is baccarat?")
answer = ask(prompt)
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      93.50 ms /   256 runs   (    0.37 ms per token,  2737.97 tokens per second)
llama_print_timings: prompt eval time =   21537.70 ms /  1750 tokens (   12.31 ms per token,    81.25 tokens per second)
llama_print_timings:        eval time =   30391.78 ms /   255 runs   (  119.18 ms per token,     8.39 tokens per second)
llama_print_timings:       total time =   52509.51 ms


" Baccarat is a popular card game played in casinos around the world. The objective of the game is for the player to predict whether the value of their hand will be closer to or equal to the value of the banker's hand. The game is usually played with eight decks of cards, and the value of each card is determined as follows: an ace counts as 1 point, a two through nine count as the face value of the card (for example, a seven counts as 7 points), and a ten or face card (king, queen) counts as 0 points. The total value of each player's hand is then calculated by adding together the values of all of the cards in that player's hand. For example, if a player was holding a two and an ace, the total value of their hand would be calculated as follows: 2 (two) + 1 (ace) = 3 points. Similarly, if a player was holding a seven and an eight, the total value of their hand would be calculated as follows: 7 (seven) + 8 (eight) = 15 points. However, since the game is usually played with eight decks of 

In [34]:
def ask_qa(user_input):
   prompt = qa_prompt_template.format(context=context_splits[0],question=user_input)
   return ask(prompt)

In [35]:
answer = ask_qa("What is Contra-d'Alembert?")
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      44.83 ms /   120 runs   (    0.37 ms per token,  2676.90 tokens per second)
llama_print_timings: prompt eval time =    1164.92 ms /    18 tokens (   64.72 ms per token,    15.45 tokens per second)
llama_print_timings:        eval time =   13307.87 ms /   119 runs   (  111.83 ms per token,     8.94 tokens per second)
llama_print_timings:       total time =   14733.31 ms


" Contra-d'Alembert is a betting system in the game of roulette. The name itself is derived from French mathematician Jean le Rond d'Alembert. \n\nThe concept behind this betting system is contrary to that of the famous martingale betting system. Unlike the martingale system, which involves increasing your bets after losing rounds, Contra-d'Alembert requires you to decrease your bets after losing rounds and increase them after winning ones. This approach aims at minimizing losses while maximizing profits."

In [45]:
answer = ask_qa('What is "Tracker Betting"?')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      31.19 ms /    84 runs   (    0.37 ms per token,  2693.26 tokens per second)
llama_print_timings: prompt eval time =    1074.32 ms /    15 tokens (   71.62 ms per token,    13.96 tokens per second)
llama_print_timings:        eval time =    8756.51 ms /    83 runs   (  105.50 ms per token,     9.48 tokens per second)
llama_print_timings:       total time =   10017.74 ms


' Tracker Betting is a betting strategy used in baccarat play. It involves tracking the results of each round of play to determine where to place bets for subsequent rounds of play. The goal of Tracker Betting is to identify patterns in the outcomes of each round of play and then use this information to make informed decisions about where to place bets in order to maximize profits and minimize losses.'

In [44]:
answer = ask_qa('Generate many questions(Q:) and answers(A:), and the answers must be contained in the above content.')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =     745.15 ms /  2048 runs   (    0.36 ms per token,  2748.45 tokens per second)
llama_print_timings: prompt eval time =    1044.97 ms /    32 tokens (   32.66 ms per token,    30.62 tokens per second)
llama_print_timings:        eval time =  334022.37 ms /  2047 runs   (  163.18 ms per token,     6.13 tokens per second)
llama_print_timings:       total time =  342075.28 ms


' A: Sure, here are some questions and answers based on the context provided:\n\nQ1: What is Target Baccarat?\nA1: Target Baccarat is a professional-level baccarat betting strategy that has never lost in over 10,000 documented games played by real players.\nQ2: How does Target Baccarat work?\nA2: Target Baccarat uses a combination of Target Betting and Tracker Betting to determine where to bet and how much to bet. The strategy also includes a Unique Bet Timing System that is used to pick the optimal time to place each bet.\n\nQ3: What are some key features of Target Baccarat?\nA3: Some key features of Target Baccarat include its ability to consistently produce explosive profits, its use of a combination of Target Betting and Tracker Betting to determine where to bet and how much to bet, its inclusion of a Unique Bet Timing System that is used to pick the optimal time to place each bet, and its overall ease of use and ability to be quickly learned and easily applied by both new and expe

In [43]:
answer = ask_qa('summarize the context')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1408.84 ms
llama_print_timings:      sample time =      33.60 ms /    89 runs   (    0.38 ms per token,  2648.49 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    9647.51 ms /    89 runs   (  108.40 ms per token,     9.23 tokens per second)
llama_print_timings:       total time =    9841.62 ms


' Target Baccarat is a professional-level baccarat strategy that has never lost. It uses Target Betting, a system borrowed from stock trading, to set up winning bets. The strategy also determines the size of bets using a Unique Betting Strategy. Overall, Target Baccarat provides an easy-to-learn and use professional-level baccarat strategy that has never lost.'

In [14]:
answer = ask_qa('how to win Big Tiger in live baccarat?')
answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1261.19 ms
llama_print_timings:      sample time =     242.53 ms /  1024 runs   (    0.24 ms per token,  4222.23 tokens per second)
llama_print_timings: prompt eval time =    7039.56 ms /   525 tokens (   13.41 ms per token,    74.58 tokens per second)
llama_print_timings:        eval time =   93114.45 ms /  1023 runs   (   91.02 ms per token,    10.99 tokens per second)
llama_print_timings:       total time =  103404.72 ms


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\